## Initialise RAG

In [2]:

from RAG.librarian import Librarian 

librarian = Librarian(librarian_LLM_model = "GEMINI")

# SELECT SPECIALIST DATABASE
librarian.select_specialist(specialist = "traveller", specialist_LLM_model = "GEMINI", )

# Ask librarian to get acquinted with the specialist database
librarian.Traveller.load_data_model(reembed = False,
                                    embed_id = 0,
                                    data_model_keys = {"TEST - CLIENT":"CLIENT ID",
                                                        "TEST - CLIENT REQUEST":"CLIENT ID",
                                                        "TEST - FLIGHTS":"FLIGHT ID",
                                                        "TEST - ACCOMODATIONS":"ACCOMODATION ID",
                                                        "TEST - ACTIVITIES":"ACTIVITY ID",
                                                        "TEST - SERVICES":"SERVICE ID",
                                                        },
                                    reembed_table = {"TEST - CLIENT":True,
                                                    "TEST - CLIENT REQUEST":True,
                                                    "TEST - FLIGHTS":True,
                                                    "TEST - ACCOMODATIONS":True,
                                                    "TEST - ACTIVITIES":True,
                                                    "TEST - SERVICES":True,
                                                    }
                                                    
                                    )


loading specialist: TRAVELLER ...
TRAVELLER embedding: TEST - CLIENT - LOADED
TRAVELLER embedding: TEST - CLIENT REQUEST - LOADED
TRAVELLER embedding: TEST - FLIGHTS - LOADED
TRAVELLER embedding: TEST - ACCOMODATIONS - LOADED
TRAVELLER embedding: TEST - ACTIVITIES - LOADED
TRAVELLER embedding: TEST - SERVICES - LOADED
TRAVELLER loaded


## _*Generate*_ : travel package from customer/agent prompt  + inventory

In [7]:
# Ask Traveller to generate a travel package
initial_query = "Hafeez from Kuala Lumpur wants to go Bali for 4 days for 2 pax. Budget: $2000. I want to chill at the beach"

convo_package = librarian.Traveller.III_generate_travel_package(initial_query = initial_query,
                                                                 topN = 6, 
                                                                 model_name = "gemini-pro",
                                                                 )

Token size of the prompt for cl100k_base ~ 3745
**Summary**

Immerse yourself in the beauty of Bali on this 4-day beach getaway designed to soothe your soul and rejuvenate your senses. Dive into the pristine waters, bask in the warm sun, indulge in luxurious spa treatments, and explore the island's stunning natural wonders. This trip_recommendation offers a perfect blend of relaxation, adventure, and cultural experiences, ensuring an unforgettable escape to paradise.

**Journey Highlights**

* Chill out on the white-sand beaches of Bali
* Pamper yourself with a rejuvenating spa treatment
* Embark on a scenic hiking and cliff-hunting tour
* Savor delicious meals at local restaurants
* Capture the beauty of Bali's rice fields
* Your journey takes you to: Kuala Lumpur - Denpasar Bali - Kuala Lumpur

**Itinerary**

**Day 1**

* 0900hrs: Depart from Kuala Lumpur International Airport (FLIGHT ID: 1)
* 1100hrs: Arrive at Denpasar Bali Airport (DPS)
* 1200hrs: Transfer to Hard Rock Hotel Bali 

## Regenerate : travel package from followup prompt + inventory

In [10]:
followup_query = "Give me the cheapest bali activities instead of that 4.5K hunting tour"

convo_package = librarian.Traveller.III_generate_travel_package(initial_query = "",
                                                                followup_query = followup_query,
                                                                 topN = 6, 
                                                                 model_name = "gemini-pro",
                                                                 )

Token size of the prompt for cl100k_base ~ 3857
**Refined Travel Package**

**Summary**

Immerse yourself in the beauty of Bali on this tailored 4-day beach getaway designed to soothe your soul and rejuvenate your senses. Dive into the pristine waters, bask in the warm sun, indulge in luxurious spa treatments, and explore the island's stunning natural wonders. This trip offers a perfect blend of relaxation, adventure, and cultural experiences, ensuring an unforgettable escape to paradise.

**Journey Highlights**

* Chill out on the white-sand beaches of Bali
* Pamper yourself with a rejuvenating spa treatment
* Embark on a scenic hiking and cliff-hunting tour
* Savor delicious meals at local restaurants
* Capture the beauty of Bali's rice fields
* Your journey takes you to: Kuala Lumpur - Denpasar Bali - Kuala Lumpur

**Itinerary**

**Day 1**

* 0900hrs: Depart from Kuala Lumpur International Airport (FLIGHT ID: 1)
* 1100hrs: Arrive at Denpasar Bali Airport (DPS)
* 1200hrs: Transfer to

## OCR : extract text from image

### a) EasyOCR(image: jpg)

In [4]:
import easyocr

def extract_text_from_image(image_file):
  """
  Extracts text and calculates average probability from an image using EasyOCR.

  Args:
      image_file: Path to the image file.

  Returns:
      A tuple containing:
          * Extracted text as a single string concatenated from all blocks.
          * Raw output from EasyOCR (list of tuples with text, probability, and bounding box).
          * Average probability of all text blocks.
  """

  # Create a reader for the languages you want to support
  reader = easyocr.Reader(['en'])

  # Use the reader to read the text from the image
  raw = reader.readtext(image_file)

  # Extract text and probabilities
  text = ' '.join([block[1] for block in raw])
  probabilities = [block[2] for block in raw]

  # Calculate average probability (handling potential division by zero)
  average_probability = sum(probabilities) / len(probabilities) if probabilities else 0.0

  return text, raw, average_probability

# Example usage
# image_file = './database/travel/Ingest/Cust_req_1.jpg'
image_path = './database/travel/Ingest/TP_Cxxx_20240105_KL1.jpg'
text, raw, avg_prob = extract_text_from_image(image_path)
print("Extracted Text:", text)
# print("Raw Data:", raw)
print("Average Probability:", avg_prob)


Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


Extracted Text: exil Ieo @ su-nig {6v.m4 One- no } | 03 2114 5414 Ajeat 54nia Trave( "V^+ I(tuc SMART Travel Agency :Sedunia Travel Services WeRLD Package Kuala Lumpur Tour Date 05 January 2024 Kuala Lumpur Tour No. of Pax 04 pax (02 adult & 02 child) Date 05 January 2024 11.OOAM 7 OOPM Full Day (8 Hours) Time Pick up Alila Hotel in Bangsar Wilayah Persekutuan Kuala Lumpur Location 58, Jalan Ang Seng, Brickfields, 50470 Kualo Lumpur, Drop off Alila Hotel in Bangsar Wilayah Persekutuan Kuala Lumpur Location 58, Jalan Ang Seng, Brickfields, 50470 Kuala Lumpur, Type of Mini Van Vehicle Tour Guide English Speaking Entrance Fee Not included Meals Adult (MYR 50.00) Child (MYR 25.00) etarian Meal for Indian family (MYR) MYR 550.00 (transport/tour) MYR 300 (Tour guide) MYR 300 (meals) MYR3OO Price (markup) Highlight Gaze at the beautiful city of Kuala Lumpur from the Observation Deck of the Petronas Twin Towers Climb Batu Caves to explore Southeast Asia'$ most visited Hindu Temple Capture your

In [5]:
text

'exil Ieo @ su-nig {6v.m4 One- no } | 03 2114 5414 Ajeat 54nia Trave( "V^+ I(tuc SMART Travel Agency :Sedunia Travel Services WeRLD Package Kuala Lumpur Tour Date 05 January 2024 Kuala Lumpur Tour No. of Pax 04 pax (02 adult & 02 child) Date 05 January 2024 11.OOAM 7 OOPM Full Day (8 Hours) Time Pick up Alila Hotel in Bangsar Wilayah Persekutuan Kuala Lumpur Location 58, Jalan Ang Seng, Brickfields, 50470 Kualo Lumpur, Drop off Alila Hotel in Bangsar Wilayah Persekutuan Kuala Lumpur Location 58, Jalan Ang Seng, Brickfields, 50470 Kuala Lumpur, Type of Mini Van Vehicle Tour Guide English Speaking Entrance Fee Not included Meals Adult (MYR 50.00) Child (MYR 25.00) etarian Meal for Indian family (MYR) MYR 550.00 (transport/tour) MYR 300 (Tour guide) MYR 300 (meals) MYR3OO Price (markup) Highlight Gaze at the beautiful city of Kuala Lumpur from the Observation Deck of the Petronas Twin Towers Climb Batu Caves to explore Southeast Asia\'$ most visited Hindu Temple Capture your memories with

### b) LLM(image: jpg)

In [7]:
image_path = './database/travel/Ingest/TP_Cxxx_20240105_KL1.jpg'
prompt_1 = """The image is a travel itinerary that i want to digitise so that i can populate a database. 
Help analyse the content and reproduce it in a structured format such that it is easily extractable
"""
response = librarian.Traveller.model_specialist.prompt_image(image_path = image_path,
                                                  prompt_1 = prompt_1,
                                                  prompt_2 = None,
                                                  model_name = "gemini-pro-vision",)

In [11]:
print(response.text)

 ```python
{
  "travel_agency": "Sedunia Travel Services",
  "package": "1 Day Kuala Lumpur Tour",
  "date": "05 January 2024",
  "no_of_pax": 4,
  "children": 2,
  "time": "11:00AM - 7:00PM",
  "duration": "8 Hours",
  "pick_up_location": "Alia Hotel in Bangsar",
  "drop_off_location": "58, Jalan Ang Seng, Brickfields, 50470 Kuala Lumpur, Wilayah Persekutuan Kuala Lumpur",
  "type_of_vehicle": "Mini Van",
  "guide": "English Speaking",
  "meals": {
    "adult": 50,
    "child": 25,
    "vegetarian_meal_for_indian_family": 30
  },
  "entrance_fee": {
    "adult": 55,
    "child": 30
  },
  "transport_fee": 30,
  "tour_guide_fee": 300,
  "total_price": 1450,
  "highlights": [
    "Gaze at the beautiful city of Kuala Lumpur from the Observation Deck of the KLCC Twin Towers",
    "Capture your memories with your camera at all thes most visited Hindu Temple in Central Market, one of the oldest markets in Kuala Lumpur",
    "Shop in Central Market",
    "Petronas Twin Towers",
    "Menara K

# 2) FM 1

In [1]:
pip install cohere

   ---------------------------------------- 0.0/151.0 kB ? eta -:--:--
   ---------------------------------------- 151.0/151.0 kB 8.8 MB/s eta 0:00:00
   ---------------------------------------- 0.0/487.4 kB ? eta -:--:--
   --------------------------------------- 487.4/487.4 kB 29.8 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [1]:
from settings import COHERE_API_KEY
import cohere
co = cohere.Client(COHERE_API_KEY)

In [2]:
from gdrive.gdrive_handler import GspreadHandler
gspread_handler = GspreadHandler(credentials_filepath='./gdrive/phrasal-ability-419201-d527372ace3b.json')
df = gspread_handler.get_sheet_as_df(sheet_name="Master Database", worksheet_name="partners")

In [13]:
df

,meta,data
0,Wang Gunung Trail Run 2.0,WANG GUNUNG\nTRAIL RUN 2.0\nCONQUER THE AGRO ...
1,Nakawan Ultra,NAKAWAUN ULTRA\nTHE THRILL OF NORTHERN TRAILS...
2,Perlis Eco X Venture,PERLIS\nECO X VENTURE \nLETS TRAVEL WITH OUR ...
3,Perlis EcoXVenture Experiences,Visit Perlis 2024 - 2025\nPerlis Indahnya Mem...
4,ClubRock Perlis 3D2N,MARK HISHAM\nGENERAL MANAGER\n\nm +6012 899 0...
5,Perlis Smart Leisure Travels,Penang MATTA Fair Special\n30.09 to 01.10.23\...
6,Perlis Eco Adventure Tours,**unicastanaholidays**\ns/bnd (KPK/LN: 1031)\...
7,Perlis Nature & Homestay Packages,PERLIS NATURE & HOMESTAY PACKAGES\n\n**Perlis...
8,Smart Leisure Travels Jom Discover Perlis,Explore \nPerlis\n<center> _Inilah Masanya_\n...
9,2D1N Harumanis Tour,MATTA FAIR\n\nSMART LEISURE TRAVELS\nSEE TRAV...


In [4]:
df
# count the number of words in each row for "data column"
df["data1"] = df["data"].apply(lambda x: x.split())
df["word_count"] = df["data1"].apply(lambda x: len(x))
# combine meta and data columns separated by | 
df["meta_data"] = df["meta"] + " | " + df["data"]
# now produce a list from meta_data column

In [3]:
# embed

In [17]:
chunks = list(df["meta_data"])

In [18]:
model="embed-english-v3.0"
response = co.embed(
    texts= chunks,
    model=model,
    input_type="search_document",
    embedding_types=['float']
)
embeddings = response.embeddings.float
print(f"We just computed {len(embeddings)} embeddings.")

We just computed 11 embeddings.


In [20]:
# store embeddings in vector database
import numpy as np
vector_database = {i: np.array(embedding) for i, embedding in enumerate(embeddings)}

User query matching to vector db

In [25]:
query = "perlis cave tour"


In [26]:
# Because the text being embedded is the search query, we set the input type as search_query
response = co.embed(
    texts=[query],
    model=model,
    input_type="search_query",
    embedding_types=['float']
)
query_embedding = response.embeddings.float[0]
print("query_embedding: ", query_embedding)

query_embedding:  [-0.015296936, -0.041290283, 0.017593384, 0.021209717, -0.038146973, 0.0036563873, 0.0015497208, -0.0041656494, 0.034362793, -0.005355835, -0.03439331, -0.01890564, -0.041900635, -0.018249512, -0.036315918, -0.045837402, 0.010765076, 0.03137207, 0.025299072, -0.00017726421, 0.0013456345, 0.0069122314, -0.0072784424, 0.003780365, 0.039611816, -0.035736084, 0.023773193, -0.008811951, 0.040496826, 0.0033416748, 0.021697998, -0.021118164, 0.01940918, -0.019760132, 0.02255249, 0.008857727, -0.00014960766, -0.014251709, 0.022018433, -0.03189087, -0.008605957, 0.018875122, -0.10681152, 0.010017395, -0.07684326, 0.0028839111, -0.08404541, -0.012649536, 0.012275696, 0.017349243, -0.0010480881, -0.0026302338, 0.010139465, -0.006591797, -0.034301758, -0.02897644, -0.017120361, 0.049224854, 0.03845215, -0.017562866, -0.040039062, 0.0068588257, 0.018188477, -0.005958557, -0.011482239, 0.0044555664, -0.0060691833, -0.011039734, 0.022827148, -0.0023555756, 0.00680542, 0.01574707, 0.

## b) Retrieve the most relevant chunks from VDB

In [28]:
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

# Calculate similarity between the user question & each chunk
similarities = [cosine_similarity(query_embedding, chunk) for chunk in embeddings]
print("similarity scores: ", similarities)

# Get indices of the top 10 most similar chunks
sorted_indices = np.argsort(similarities)[::-1]

# Keep only the top 10 indices
top_indices = sorted_indices[:5]
print("Here are the indices of the top 10 chunks after retrieval: ", top_indices)

# Retrieve the top 10 most similar chunks
top_chunks_after_retrieval = [chunks[i] for i in top_indices]
print("Here are the top 10 chunks after retrieval: ")
for t in top_chunks_after_retrieval:
    print("== " + t)

similarity scores:  [0.32357911627988006, 0.4621148676947522, 0.6483553170880503, 0.6977026551801692, 0.5742969324206351, 0.5693645202466326, 0.6401380773195966, 0.645026794678863, 0.6367969414522532, 0.4436916115230696, 0.44847051140846783]
Here are the indices of the top 10 chunks after retrieval:  [3 2 7 6 8]
Here are the top 10 chunks after retrieval: 
== Perlis EcoXVenture Experiences |  Visit Perlis 2024 - 2025
Perlis Indahnya Memang Menawan

Book Now
Perlis Ecoventure Experiences

Caving Exploration
Explore the nature of the millions years limestone cave
Location: Gua Wang Burma / Gua Wang Buluh
Duration: 2 Hours
RM220.00 / USD55 Per Person

Sunrise Trail
Explore the beautiful panorama view of sunrise in Perlis
Location: Wang Kelian Viewpoint, Kuala Bukit
Duration: 2 Hours
RM180.00 / USD45.00 Per Person

Sunset Experience
Experience beautiful sunset view in Perlis
Location: Kuala Perlis / Kg. Warna Warni
Duration: 2 Hours
RM180.00 / USD45.00 Per Person

Photo Clicking
Revealing 

## c) Rerank 
Check relevance of this

In [33]:
response = co.rerank(
    query=query,
    documents=top_chunks_after_retrieval,
    top_n=3,
    model="rerank-english-v2.0",
)

# top_chunks_after_rerank = [result.document['text'] for result in response]
# print("Here are the top 3 chunks after rerank: ")
# for t in top_chunks_after_rerank:
#     print("== " + t)

In [36]:
top_chunks_after_retrieval[0]

'Perlis EcoXVenture Experiences |  Visit Perlis 2024 - 2025\nPerlis Indahnya Memang Menawan\n\nBook Now\nPerlis Ecoventure Experiences\n\nCaving Exploration\nExplore the nature of the millions years limestone cave\nLocation: Gua Wang Burma / Gua Wang Buluh\nDuration: 2 Hours\nRM220.00 / USD55 Per Person\n\nSunrise Trail\nExplore the beautiful panorama view of sunrise in Perlis\nLocation: Wang Kelian Viewpoint, Kuala Bukit\nDuration: 2 Hours\nRM180.00 / USD45.00 Per Person\n\nSunset Experience\nExperience beautiful sunset view in Perlis\nLocation: Kuala Perlis / Kg. Warna Warni\nDuration: 2 Hours\nRM180.00 / USD45.00 Per Person\n\nPhoto Clicking\nRevealing the beauty of Perlis nature\nLocation: 3 - 4 Hours\nDuration: 2 Hours\nRM220 / USD55.00 Per Person\n\nCall for more information\n012 - 476 7743\n\nDay Trip Packages\n\nPACKAGE INCLUDES: TERM & CONDITION:\nMain Activities\nTransportation\nDriver\n1 x Meal\nLocal Guide / Crew\n\nMinimum of 4 pax\nValid until Dec 2024\nTravelling period 

In [34]:
for result in response:
    print(result)

('id', '92f6def0-8783-4c6d-ad1f-9cbf33933ce1')
('results', [RerankResponseResultsItem(document=None, index=3, relevance_score=0.99858963), RerankResponseResultsItem(document=None, index=0, relevance_score=0.99827254), RerankResponseResultsItem(document=None, index=2, relevance_score=0.99770385)])
('meta', ApiMeta(api_version=ApiMetaApiVersion(version='1', is_deprecated=None, is_experimental=None), billed_units=ApiMetaBilledUnits(input_tokens=None, output_tokens=None, search_units=1, classifications=None), tokens=None, warnings=None))


## d) Generate final model response

In [37]:
# preamble containing instructions about the task and the desired style for the output.
preamble = """
## Task & Context
You help people answer their questions and other requests interactively. You will be asked a very wide array of requests on all kinds of topics. You will be equipped with a wide range of search engines or similar tools to help you, which you use to research your answer. You should focus on serving the user's needs as best you can, which will be wide-ranging.

## Style Guide
Unless the user asks for a different style of answer, you should answer in full sentences, using proper grammar and spelling.
"""

In [38]:
# retrieved documents
documents = [
    {"title": "chunk 0", "snippet": top_chunks_after_retrieval[0]},
    {"title": "chunk 1", "snippet": top_chunks_after_retrieval[1]},
    {"title": "chunk 2", "snippet": top_chunks_after_retrieval[2]},
  ]

# get model response
response = co.chat(
  message=query,
  documents=documents,
  preamble=preamble,
  model="command-r",
  temperature=0.3
)

print("Final answer:")
print(response.text)

Final answer:
There are a number of caving tours available in Perlis, Malaysia.

Perlis EcoXVenture Experiences offer a two-hour caving exploration of Gua Wang Burma or Gua Wang Buluh caves for RM220.00/USD55 per person.

Unicastana Holidays offer several package deals including Extreme Gateway, Touch The Sky and Green Paradise which also involve caving and have prices starting from RM499 for 3 days and 2 nights. 

The Perlis Cycling Tour is a 3-day, 2-night tour focusing on cycling, with additional activities, priced from RM339.


In [1]:
from RAG.cohere_RAG import RAG

rag = RAG()
result = rag.rag_pipeline(query = "perlis cave tour", sheet_name = "Master Database", worksheet_name="partners")

We just computed 11 embeddings.
query_embedding:  [-0.015296936, -0.041290283, 0.017593384, 0.021209717, -0.038146973, 0.0036563873, 0.0015497208, -0.0041656494, 0.034362793, -0.005355835, -0.03439331, -0.01890564, -0.041900635, -0.018249512, -0.036315918, -0.045837402, 0.010765076, 0.03137207, 0.025299072, -0.00017726421, 0.0013456345, 0.0069122314, -0.0072784424, 0.003780365, 0.039611816, -0.035736084, 0.023773193, -0.008811951, 0.040496826, 0.0033416748, 0.021697998, -0.021118164, 0.01940918, -0.019760132, 0.02255249, 0.008857727, -0.00014960766, -0.014251709, 0.022018433, -0.03189087, -0.008605957, 0.018875122, -0.10681152, 0.010017395, -0.07684326, 0.0028839111, -0.08404541, -0.012649536, 0.012275696, 0.017349243, -0.0010480881, -0.0026302338, 0.010139465, -0.006591797, -0.034301758, -0.02897644, -0.017120361, 0.049224854, 0.03845215, -0.017562866, -0.040039062, 0.0068588257, 0.018188477, -0.005958557, -0.011482239, 0.0044555664, -0.0060691833, -0.011039734, 0.022827148, -0.002355

In [4]:
print(result)

There are a number of caving tours available in Perlis, Malaysia.

Perlis EcoXVenture Experiences offer a two-hour caving exploration of Gua Wang Burma or Gua Wang Buluh caves for RM220.00/USD55 per person.

Unicastana Holidays offer several package tours that include caving. Their Extreme Gateway, Touch The Sky, Green Paradise and Back To Nature packages all feature caving activities and are priced from RM499 for three days and two nights. Some of the packages also include rock climbing, hiking, and birdwatching.
